In [67]:
import numpy as np
from scipy.signal import find_peaks
from scipy.stats import kurtosis,skew
import pywt
import antropy as ant  # for sample entropy
from sklearn.impute import KNNImputer
from tqdm import tqdm
import neurokit2 as nk
import scipy.signal as ssignal

import pickle

In [68]:
# Data Loading
pickle_path = 'data_quicksafe.pkl'

with open(pickle_path, 'rb') as f:
    Xtrain, ytrain, Xval, yval = pickle.load(f)
print("Data loaded from pickle file.")


Data loaded from pickle file.


In [69]:
to_dispose = [1885, 3427, 3546, 5074, 6924, 7103, 7102, 8140]
#Xtrain = np.delete(Xtrain, to_dispose, axis=0)
#ytrain = np.delete(ytrain, to_dispose, axis=0)

to_dispose_val = [916]
#Xval = np.delete(Xval, to_dispose_val, axis=0)
#yval = np.delete(yval, to_dispose_val, axis=0)

In [70]:
fs=250
Xtrain.shape

(10253, 5000, 5)

In [71]:
# Combine train and val for feature selection
#dataIn = np.concatenate((Xtrain, Xval), axis=0)
dataIn = Xtrain
#dataInY = np.concatenate((ytrain, yval), axis=0)
dataInY = ytrain

In [72]:
def detect_abp_syst_peaks(signal):
    fs = 250
    peaks, _ = find_peaks(signal, distance=int(0.3*fs), height=signal.mean()+5)
    peaks_inverse, _ = find_peaks(-signal, distance=int(0.3*fs), height=-signal.mean()+5,)
    peaks_all = peaks_inverse

    # Filter Diastolic peaks by removing all but the first one after the peak foot
    for i in range(len(peaks)-1):
        mask = (peaks_all > peaks[i]) & (peaks_all < peaks[i+1])
        if np.sum(mask) > 1:
            to_remove = peaks_all[mask][1:]
            peaks_all = peaks_all[~np.isin(peaks_all, to_remove)]

    abp_diastolic_new = []
    for i in range(len(peaks)-1):
        mask = (peaks_inverse > peaks[i]) & (peaks_inverse < peaks[i+1])
        if np.sum(mask) > 0:
            to_keep = peaks_inverse[mask]
            closest = to_keep[np.argmin(peaks[i+1] - to_keep)]
            abp_diastolic_new.append(closest)

    abp_diastolic_new = np.array(abp_diastolic_new)

    return peaks, peaks_inverse, abp_diastolic_new

peaks = []
for sample in dataIn:
    signal = sample[:,0]
    syst_peaks, dia_peaks, distolic_peaks = detect_abp_syst_peaks(signal)
    ppg = sample[:,3]
    ppg_peaks, _ = find_peaks(ppg, distance=int(0.3*fs), height=ppg.mean()+3)

    if len(distolic_peaks) == 0:
        placeholder = np.zeros(3, dtype=np.float32)
        placeholder_int = np.zeros(3, dtype=np.int32)
        peaks.append((placeholder_int, placeholder_int, placeholder_int, placeholder, placeholder, placeholder, ppg_peaks))
        continue

    syst_peaks_values = signal[syst_peaks]
    diatronic_peaks_values = signal[dia_peaks]
    distolic_peaks_values = signal[distolic_peaks]


    peaks.append((syst_peaks, dia_peaks, distolic_peaks, syst_peaks_values, diatronic_peaks_values, distolic_peaks_values, ppg_peaks))

In [73]:
from scipy.signal import butter, filtfilt
import pandas as pd
import scipy.signal as signal
from scipy.fft import fft, fftfreq

def bandpass_filter(signal, lowcut=0.5, highcut=40.0, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

def ecg_bandpass(ecg_signal):
    return bandpass_filter(ecg_signal, lowcut=0.5, highcut=40.0, order=4)


def get_pp(signal, signal_id):
    systolic = np.array(peaks[signal_id][0])
    diastolic = np.array(peaks[signal_id][2])
    time_diffs = []
    pp_values = []
    for i in range(len(systolic)-1):
        mask = (diastolic > systolic[i]) & (diastolic < systolic[i+1])

        if np.sum(mask) > 0:
            diastolic_peak = diastolic[mask][0]
            pp = signal[systolic[i]] - signal[diastolic_peak]
            time_diff = (diastolic_peak - systolic[i]) / fs
            time_diffs.append(time_diff)
            pp_values.append(pp)

    return pp_values, time_diffs

def get_wavelet_entropies(signal):
    wavelet_features = []
    coeffs = pywt.wavedec(signal, 'bior3.3', level=5)

    for subband in coeffs:
        # Sample entropy
        sampen = ant.sample_entropy(subband)
        # Wavelet entropy (Shannon entropy of normalized coefficient energy)
        e = subband**2
        p = e / np.sum(e)
        wavelet_entropy = -np.sum(p * np.log2(p + 1e-12))
        wavelet_features.extend([sampen, wavelet_entropy])

    return wavelet_features

def get_abp_hr(signal_id):
    hr_values = []
    syst_peaks = peaks[signal_id][0]
    for i in range(1, len(syst_peaks)):
        left_time = (syst_peaks[i] - syst_peaks[i-1]) / fs
        hr = 60 /  left_time
        hr_values.append(hr)

    return hr_values

def get_ptt(signal_id):
    ptt_values = []
    syst_peaks = peaks[signal_id][0]
    ppg_peaks = peaks[signal_id][6]
    for sp in syst_peaks:
        ppg_candidates = ppg_peaks[ppg_peaks > sp]
        if len(ppg_candidates) > 0:
            ptt = (ppg_candidates[0] - sp) / fs
            ptt_values.append(ptt)

    return ptt_values

def compute_hrv_freq(ecg_signal):
    # R-peak detection (simple, replace with robust detector in practice)
    peaks, _ = ssignal.find_peaks(ecg_signal, distance=int(0.25*fs))  # ensure integer distance
    if len(peaks) < 2:
        return np.nan, np.nan, np.nan

    rr_intervals = np.diff(peaks) / fs  # in seconds

    # Interpolate RR intervals to evenly sampled series
    t_rr = np.cumsum(rr_intervals)
    t_interp = np.linspace(0, t_rr[-1], len(rr_intervals)*4)
    rr_interp = np.interp(t_interp, t_rr, rr_intervals)

    # Welch PSD (set nperseg <= length to avoid warnings)
    nperseg = min(256, len(rr_interp))
    f, pxx = ssignal.welch(rr_interp - np.mean(rr_interp), fs=4.0, nperseg=nperseg)

    # Bands
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    lf_mask = (f >= lf_band[0]) & (f < lf_band[1])
    hf_mask = (f >= hf_band[0]) & (f < hf_band[1])

    lf_power = np.trapezoid(pxx[lf_mask], f[lf_mask]) if np.any(lf_mask) else np.nan
    hf_power = np.trapezoid(pxx[hf_mask], f[hf_mask]) if np.any(hf_mask) else np.nan
    lf_hf_ratio = lf_power / hf_power if hf_power and hf_power > 0 else np.nan

    return lf_power, hf_power, lf_hf_ratio

# 2. PPG Harmonic Ratio (H2/H1)
def compute_ppg_harmonic_ratio(ppg_signal):
    # Segment a single beat (naive: first ~1 sec)
    segment = ppg_signal[:fs]
    N = len(segment)
    if N < 2:
        return np.nan

    yf = np.abs(fft(segment))[:N//2]
    xf = fftfreq(N, 1/fs)[:N//2]

    # Find fundamental frequency (peak in 0.5–3 Hz, typical HR)
    mask = (xf >= 0.5) & (xf <= 3)
    if not np.any(mask):
        return np.nan

    fundamental_idx = np.argmax(yf[mask]) + np.where(mask)[0][0]
    f0 = xf[fundamental_idx]

    # Amplitude at fundamental and 2nd harmonic
    H1 = yf[fundamental_idx]
    H2_idx = np.argmin(np.abs(xf - 2*f0))
    H2 = yf[H2_idx]

    return H2 / H1 if H1 > 0 else np.nan


# ---------------------------
# 3. ABP Spectral Centroid
# ---------------------------
def compute_abp_spectral_centroid(abp_signal):
    nperseg = min(1024, len(abp_signal))
    f, pxx = ssignal.welch(abp_signal - np.mean(abp_signal), fs=fs, nperseg=nperseg)
    if np.sum(pxx) == 0:
        return np.nan
    centroid = np.sum(f * pxx) / np.sum(pxx)
    return centroid


accumulation_types = {
    'mean': np.mean,
    'std': np.std,
    'min': np.min,
    'max': np.max,
    'kurtosis': kurtosis,
    'skewness': skew,
    'median': np.median,
    'sum': np.sum,
    'var': np.var,
    'var_coeff': lambda x: ((np.std(x))/(np.mean(x)+1e-6)) * 100,
}

def build_feature_table(data_in):
    
    raw_features = {
        'abp_syst_peaks': lambda signal_id: peaks[signal_id][3],
        'abp_ditr_peaks': lambda signal_id: peaks[signal_id][4],
        'abp_dist_peaks': lambda signal_id: peaks[signal_id][5],
        'pulse_pressure': lambda signal_id: get_pp(data_in[signal_id,:,0], signal_id)[0],
        'pulse_transit_time': lambda signal_id: get_ptt(signal_id),
        'syst_dist_time_diff': lambda signal_id: get_pp(data_in[signal_id,:,0], signal_id)[1],
        'abp_hr': lambda signal_id: get_abp_hr(signal_id),
        'ecg_hr': lambda signal_id: nk.ecg_rate(nk.ecg_findpeaks(ecg_bandpass(data_in[signal_id,:,1]), sampling_rate=fs)['ECG_R_Peaks'], sampling_rate=fs),
        #'map': lambda signal_id: peaks[signal_id] + [v/3 for v in get_pp(data_in[signal_id,:,0], signal_id)[0]],

        'raw_ecgii': lambda signal_id: ecg_bandpass(data_in[signal_id,:,1]),
        'raw_abp': lambda signal_id: data_in[signal_id,:,0],
        'raw_ppg': lambda signal_id: data_in[signal_id,:,3],
        'raw_ecgv': lambda signal_id: ecg_bandpass(data_in[signal_id,:,2]),
    }

    raw_features_no_accum = {
        'wavelet_entropies_ecgii': lambda signal_id: get_wavelet_entropies(data_in[signal_id,:,1]),
        'wavelet_entropies_ecgii_filtered': lambda signal_id: get_wavelet_entropies(ecg_bandpass(data_in[signal_id,:,1])),
        'wavelet_entropies_ecgv': lambda signal_id: get_wavelet_entropies(ecg_bandpass(data_in[signal_id,:,2])),
        'wavelet_entropies_abp': lambda signal_id: get_wavelet_entropies(data_in[signal_id,:,0]),
        'wavelet_entropies_ppg': lambda signal_id: get_wavelet_entropies(data_in[signal_id,:,3]),

        'ecg_lf_power': lambda signal_id: compute_hrv_freq(ecg_bandpass(data_in[signal_id,:,1]))[0],
        'ecg_hf_power': lambda signal_id: compute_hrv_freq(ecg_bandpass(data_in[signal_id,:,1]))[1],
        'ecg_lf_hf_ratio': lambda signal_id: compute_hrv_freq(ecg_bandpass(data_in[signal_id,:,1]))[2],
        'ecg_lf_hf_ratio_v5': lambda signal_id: compute_hrv_freq(ecg_bandpass(data_in[signal_id,:,2]))[2],
        'ppg_h2_h1_ratio': lambda signal_id: compute_ppg_harmonic_ratio(data_in[signal_id,:,3]),
        'abp_spectral_centroid': lambda signal_id: compute_abp_spectral_centroid(data_in[signal_id,:,0]),
        'ppg_spectral_centroid': lambda signal_id: compute_abp_spectral_centroid(data_in[signal_id,:,3]),
    }
    
    feature_table = []
    for signal_id in tqdm(range(data_in.shape[0]), desc="Extracting features"):
        feature_vector = {}
        for feature_name, func in raw_features.items():
            feature_data = func(signal_id)

            for acc_name, acc_func in accumulation_types.items():
                try:
                    acc_value = acc_func(feature_data)
                    if acc_value is None or (isinstance(acc_value, float) and (np.isnan(acc_value) or np.isinf(acc_value))):
                        print(
                            f"Warning: Feature {feature_name} for signal {signal_id} produced invalid value {acc_value} with accumulation {acc_name}"
                        )
                except Exception as e:
                    acc_value = np.nan
                    print(f"Error computing {acc_name} for feature {feature_name} on signal {signal_id}: {e}")

                feature_vector[f"{feature_name}_{acc_name}"] = acc_value



        for feature_name, func in raw_features_no_accum.items():
            feature_data = func(signal_id)
            if isinstance(feature_data, (list, np.ndarray)):
                for i, value in enumerate(feature_data):
                    if value is None or (isinstance(value, float) and (np.isnan(value) or np.isinf(value))):
                        print(
                            f"Warning: Feature {feature_name} for signal {signal_id} produced invalid value {value} at index {i}"
                        )
                    feature_vector[f"{feature_name}_{i}"] = value
            else:
                if feature_data is None or (isinstance(feature_data, float) and (np.isnan(feature_data) or np.isinf(feature_data))):
                    print(
                        f"Warning: Feature {feature_name} for signal {signal_id} produced invalid value {feature_data}"
                    )
                feature_vector[feature_name] = feature_data
        feature_table.append(feature_vector)

    df = pd.DataFrame(feature_table)
    return df

In [74]:
df = build_feature_table(dataIn)
dataInProcessed = df.copy()
df

Extracting features:  18%|█▊        | 1884/10253 [01:06<05:01, 27.76it/s]/Users/U465136/venvs/torch_312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/U465136/venvs/torch_312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/U465136/venvs/torch_312/lib/python3.12/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/U465136/venvs/torch_312/lib/python3.12/site-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/U465136/venvs/torch_312/lib/python3.12/site-packages/numpy/_core/_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.t

Error computing min for feature pulse_pressure on signal 1885: zero-size array to reduction operation minimum which has no identity
Error computing max for feature pulse_pressure on signal 1885: zero-size array to reduction operation maximum which has no identity
Error computing min for feature syst_dist_time_diff on signal 1885: zero-size array to reduction operation minimum which has no identity
Error computing max for feature syst_dist_time_diff on signal 1885: zero-size array to reduction operation maximum which has no identity


Extracting features:  20%|█▉        | 2005/10253 [01:10<04:45, 28.91it/s]

Extracting features:  33%|███▎      | 3433/10253 [02:01<03:49, 29.74it/s]

Error computing min for feature pulse_pressure on signal 3427: zero-size array to reduction operation minimum which has no identity
Error computing max for feature pulse_pressure on signal 3427: zero-size array to reduction operation maximum which has no identity
Error computing min for feature syst_dist_time_diff on signal 3427: zero-size array to reduction operation minimum which has no identity
Error computing max for feature syst_dist_time_diff on signal 3427: zero-size array to reduction operation maximum which has no identity


...

Extracting features:  83%|████████▎ | 8495/10253 [04:59<01:00, 28.86it/s]

Extracting features:  83%|████████▎ | 8513/10253 [05:00<01:00, 28.91it/s]

Extracting features:  95%|█████████▌| 9751/10253 [05:45<00:18, 27.10it/s]

Extracting features: 100%|██████████| 10253/10253 [06:02<00:00, 28.30it/s]


,abp_syst_peaks_mean,abp_syst_peaks_std,abp_syst_peaks_min,abp_syst_peaks_max,abp_syst_peaks_kurtosis,abp_syst_peaks_skewness,abp_syst_peaks_median,abp_syst_peaks_sum,abp_syst_peaks_var,abp_syst_peaks_var_coeff,...,wavelet_entropies_ppg_9,wavelet_entropies_ppg_10,wavelet_entropies_ppg_11,ecg_lf_power,ecg_hf_power,ecg_lf_hf_ratio,ecg_lf_hf_ratio_v5,ppg_h2_h1_ratio,abp_spectral_centroid,ppg_spectral_centroid
0,71.075584,1.164184,69.199402,73.149200,-1.010610,0.194026,71.174301,1421.511673,1.355325,1.637953,...,9.138704,1.372074,10.112689,0.000003,0.002633,0.001183,0.056487,0.643355,1.785233,1.506840
1,69.254283,1.414661,67.224503,72.161797,-0.853037,0.469304,69.199402,1246.577095,2.001266,2.042706,...,9.111947,1.382688,10.107423,0.000026,0.001750,0.014754,0.172541,0.628875,1.781783,1.516306
2,69.459279,1.102469,67.224503,71.174301,-0.790912,-0.077322,69.199402,1319.726295,1.215438,1.587217,...,9.216452,1.391516,10.117354,0.000051,0.001548,0.032907,1.264350,0.640112,1.830099,1.498332
3,70.878089,1.252935,69.199402,73.149200,-1.027206,0.281935,70.680599,1417.561783,1.569846,1.767732,...,9.175560,1.404966,10.152922,0.000016,0.001482,0.010938,0.177763,0.589374,1.767455,1.422974
4,73.478384,1.316625,71.174301,76.111603,-0.539106,0.078147,73.149200,1322.610909,1.733503,1.791854,...,9.149633,1.336129,10.090768,0.000405,0.001806,0.224448,0.541504,0.594130,1.686234,1.435762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10248,115.337449,2.104992,111.660004,119.559998,-0.778623,0.105756,115.610001,3344.786026,4.430991,1.825072,...,9.176041,1.000391,10.019302,0.000258,0.001956,0.131957,0.157640,0.171063,2.129478,1.215260
10249,115.116108,2.293038,111.660004,118.571999,-1.317916,0.136940,114.622002,3223.251030,5.258024,1.991935,...,9.194430,1.105939,10.094158,0.000076,0.000912,0.083370,0.060247,0.124082,2.151763,1.008254
10250,113.600898,2.237886,109.684998,117.584999,-1.262777,0.113778,113.635002,3294.426033,5.008135,1.969955,...,9.201999,1.180739,10.179529,0.000354,0.000943,0.375317,0.121741,0.438144,2.172959,0.617252
10251,108.368573,6.993245,87.961098,123.510002,1.620491,-0.598908,108.697998,3251.057198,48.905476,6.453204,...,9.106087,0.951534,10.084895,0.001751,0.002208,0.792797,0.363965,0.629820,2.250588,0.941969


In [76]:
df

,abp_syst_peaks_mean,abp_syst_peaks_std,abp_syst_peaks_min,abp_syst_peaks_max,abp_syst_peaks_kurtosis,abp_syst_peaks_skewness,abp_syst_peaks_median,abp_syst_peaks_sum,abp_syst_peaks_var,abp_syst_peaks_var_coeff,...,wavelet_entropies_ppg_9,wavelet_entropies_ppg_10,wavelet_entropies_ppg_11,ecg_lf_power,ecg_hf_power,ecg_lf_hf_ratio,ecg_lf_hf_ratio_v5,ppg_h2_h1_ratio,abp_spectral_centroid,ppg_spectral_centroid
0,71.075584,1.164184,69.199402,73.149200,-1.010610,0.194026,71.174301,1421.511673,1.355325,1.637953,...,9.138704,1.372074,10.112689,0.000003,0.002633,0.001183,0.056487,0.643355,1.785233,1.506840
1,69.254283,1.414661,67.224503,72.161797,-0.853037,0.469304,69.199402,1246.577095,2.001266,2.042706,...,9.111947,1.382688,10.107423,0.000026,0.001750,0.014754,0.172541,0.628875,1.781783,1.516306
2,69.459279,1.102469,67.224503,71.174301,-0.790912,-0.077322,69.199402,1319.726295,1.215438,1.587217,...,9.216452,1.391516,10.117354,0.000051,0.001548,0.032907,1.264350,0.640112,1.830099,1.498332
3,70.878089,1.252935,69.199402,73.149200,-1.027206,0.281935,70.680599,1417.561783,1.569846,1.767732,...,9.175560,1.404966,10.152922,0.000016,0.001482,0.010938,0.177763,0.589374,1.767455,1.422974
4,73.478384,1.316625,71.174301,76.111603,-0.539106,0.078147,73.149200,1322.610909,1.733503,1.791854,...,9.149633,1.336129,10.090768,0.000405,0.001806,0.224448,0.541504,0.594130,1.686234,1.435762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10248,115.337449,2.104992,111.660004,119.559998,-0.778623,0.105756,115.610001,3344.786026,4.430991,1.825072,...,9.176041,1.000391,10.019302,0.000258,0.001956,0.131957,0.157640,0.171063,2.129478,1.215260
10249,115.116108,2.293038,111.660004,118.571999,-1.317916,0.136940,114.622002,3223.251030,5.258024,1.991935,...,9.194430,1.105939,10.094158,0.000076,0.000912,0.083370,0.060247,0.124082,2.151763,1.008254
10250,113.600898,2.237886,109.684998,117.584999,-1.262777,0.113778,113.635002,3294.426033,5.008135,1.969955,...,9.201999,1.180739,10.179529,0.000354,0.000943,0.375317,0.121741,0.438144,2.172959,0.617252
10251,108.368573,6.993245,87.961098,123.510002,1.620491,-0.598908,108.697998,3251.057198,48.905476,6.453204,...,9.106087,0.951534,10.084895,0.001751,0.002208,0.792797,0.363965,0.629820,2.250588,0.941969


In [77]:
def zero_outliers_iqr(df, columns=None, factor=20.0):
    """
    Replaces extreme outliers with 0.0 using IQR method.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    columns : list, optional
        Columns to check for outliers. If None, all numeric columns are used.
    factor : float, optional
        IQR factor. Default=3.0 (conservative).

    Returns:
    --------
    pd.DataFrame
        Dataframe with outliers replaced by 0.0
    """
    df = df.copy()
    if columns is None:
        columns = df.select_dtypes(include=np.number).columns

    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - factor * IQR
        upper = Q3 + factor * IQR

        # replace values outside bounds with 0.0
        df.loc[(df[col] < lower) | (df[col] > upper), col] = 0.0

    return df

In [78]:
# Remove outliers
#dfNew = zero_outliers_iqr(df, factor=10.0)

In [79]:
# get amount of values that are greater than 999999
#print((dfNew > 999999).sum().sum())

0


In [80]:
#dfNew = zero_outliers_iqr(df, factor=10.0)

#print((dfNew > 999999).sum().sum())

# DF check for nan values
#print(df.isna().sum().sum())
dfFilled = df.copy().fillna(0.0)
dfFilled.isna().sum().sum()

# Remove inf
dfFilled.replace([np.inf, -np.inf], 0.0, inplace=True)


354


In [81]:
feature_name_list = df.columns.tolist()
dataInProcessed = dfFilled.copy()
dataInProcessedNumpy = dfFilled.to_numpy()
# To Datatype float32
dataInProcessedNumpy = dataInProcessedNumpy.astype(np.float32)
dataInProcessedNumpy.shape

(10253, 187)

In [82]:
dataInY = dataInY.astype(np.bool)
dataInY.shape

# Scaling the DataInY like in the model.py
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataInProcessed = scaler.fit_transform(dataInProcessedNumpy)
dataInProcessed.shape
dataInProcessed

array([[-2.368851  , -0.77304995, -1.7664517 , ...,  0.7575857 ,
        -0.4008075 , -0.06997653],
       [-2.4564543 , -0.7065524 , -1.8499416 , ...,  0.6891004 ,
        -0.4084342 , -0.04623661],
       [-2.446594  , -0.7894343 , -1.8499416 , ...,  0.74224985,
        -0.3015975 , -0.09131358],
       ...,
       [-0.323416  , -0.4879996 , -0.05490065, ..., -0.21304473,
         0.45654482, -2.301059  ],
       [-0.5750867 ,  0.7744706 , -0.97329056, ...,  0.69356585,
         0.6282005 , -1.4866695 ],
       [ 0.96852833,  3.1332135 ,  0.11208799, ...,  1.0851644 ,
        -0.7942545 , -1.4639955 ]], shape=(10253, 187), dtype=float32)

In [83]:
%%javascript
(function(on) {
    const e = $("<a>Setup failed</a>");
    const ns = "js_jupyter_suppress_warnings";
    var cssrules = $("#" + ns);
    if(!cssrules.length)
        cssrules = $("<style id='" + ns + "' type='text/css'>div.output_stderr { } </style>").appendTo("head");
    e.click(function() {
        var s = 'Showing';
        cssrules.empty()
        if(on) {
            s = 'Hiding';
            cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
        }
        e.text(s + ' warnings (click to toggle)');
        on = !on;
    }).click();
    $(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [84]:
from sklearn_genetic import GAFeatureSelectionCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve, auc,classification_report,confusion_matrix

def test_evaluation(estimator, x, y):
    yproba = estimator.predict_proba(x)[:, 1]

    test_prc, test_rec, thresholds = precision_recall_curve(y, yproba)
    test_auprc = auc(test_rec, test_prc)
    return test_auprc

# Optimization
estimator = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
)

selector = GAFeatureSelectionCV(
    estimator,
    cv=5,                          # cross-validation folds
    verbose=1,
    scoring=test_evaluation,#'roc_auc_ovo', #test_evaluation,            # optimize for accuracy
    max_features=50,       # allow up to all features
    population_size=50,               # GA population size
    crossover_probability=0.3,
    mutation_probability=0.5,
    generations=50,              # number of generations
    n_jobs=-1
)
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    selector.fit(dataInProcessed, dataInY)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for 

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	-31999.8	46647.7    	0.287522   	-100000    
1  	84    	-13999.8	34698.8    	0.287522   	-100000    
2  	79    	-5999.75	23748.7    	0.303213   	-100000    
3  	81    	-7999.75	27129.4    	0.308534   	-100000    
4  	86    	-3999.73	19596      	0.308534   	-100000    
5  	87    	-7999.74	27129.4    	0.308534   	-100000    
6  	85    	-5999.73	23748.8    	0.308534   	-100000    
7  	73    	-3999.72	19596      	0.308906   	-100000    
8  	81    	-5999.72	23748.8    	0.308906   	-100000    
9  	79    	-5999.72	23748.8    	0.318858   	-100000    
10 	80    	-5999.71	23748.8    	0.322533   	-100000    
11 	78    	-1999.7 	14000      	0.320649   	-100000    
12 	79    	-9999.72	30000.1    	0.320649   	-100000    
13 	79    	-5999.71	23748.8    	0.326874   	-100000    
14 	79    	-7999.71	27129.4    	0.326874   	-100000    
15 	83    	-5999.71	23748.8    	0.326874   	-100000    
16 	80    	-5999.7 	23748.8    	0.340875   	-100

In [85]:

# -----------------------------
# Inspect Results
# -----------------------------
print(selector.best_features_)

selected_mask = selector.support_

# Columns print
selected_features = [feature_name_list[i] for i in range(len(feature_name_list)) if selected_mask[i]]
print("Selected features:")
print(selected_features)
print(len(selected_features))

[False False False False False False False False False False  True False
 False  True  True False False False  True False  True  True False  True
  True False False False  True  True False False False False False False
 False False False  True False False  True False  True False  True False
 False  True False False False False False False False False False False
  True False False  True  True False False False  True False False False
 False False False False False False False  True False  True False False
 False False False False False  True  True False False False False False
 False False False False  True False False False False False False False
 False False False False  True False False False False False  True False
 False  True False False False False False False False False False False
  True False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [27]:
from pickle import dump,load

# save Estimtor, Selector
with open('xgb_estimator.pkl', 'wb') as f:
    dump(estimator, f)

with open('ga_selector.pkl', 'wb') as f:
    dump(selector, f)

# Save selected features
with open('selected_features.pkl', 'wb') as f:
    dump(selected_features, f)
